## Sobre la base de datos

Está relacionada al trabajo:
[SATPdb: a database of structurally annotated therapeutic peptides](hhttps://pmc.ncbi.nlm.nih.gov/articles/PMC4702810/)

Contiene registros positivos de péptidos antifúngicos.

Usaremos la biblioteca `Biopython` que es muy útil para trabajar con secuencias biológicas.

In [1]:
%pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.9 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import pandas as pd

In [3]:
fasta_pos = '/content/antifungal.fasta'

## Secuencias positivas

In [4]:
data = []
try:
    for record in SeqIO.parse(fasta_pos, "fasta"):
        data.append({
            "ID": record.id,
            "Description": record.description,
            "Sequence": str(record.seq)
        })

    df_pos = pd.DataFrame(data)
    display(df_pos.head())

except FileNotFoundError:
    print(f"Error: El archivo no se encuentra en la ruta especificada: {fasta_pos}")
except Exception as e:
    print(f"Ocurrió un error al parsear el archivo fasta: {e}")

,ID,Description,Sequence
0,satpdb10005,satpdb10005,KWKKLLKKPPPLLKKLLKKL
1,satpdb10030,satpdb10030,GIWDTIKSMGKVFAGKILQN
2,satpdb10043,satpdb10043,VIPFVASVAAEMMQHVYCAASKKC
3,satpdb10051,satpdb10051,ILGPVISTIGGVLGGLLKNL
4,satpdb10077,satpdb10077,FLWGLIPGAISAVTSLIKK


In [6]:
# Seleccionar las secuencias positivas
df = df_pos[['Sequence']]

# Agregar una columna de label (si tiene actividad antifúngica o no)
df['label'] = 1

/tmp/ipython-input-4057750613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = 1


In [8]:
df.shape

(1412, 2)

## Verificar duplicados

In [9]:
df["Sequence"].unique().shape

(1328,)

In [10]:
df = df.drop_duplicates().copy()

In [11]:
df.shape

(1328, 2)

## Verificar canónicos

In [12]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [13]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["Sequence"]]

In [14]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 53


In [15]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 1275


In [16]:
display(df_validas)

,Sequence,label
0,KWKKLLKKPPPLLKKLLKKL,1
1,GIWDTIKSMGKVFAGKILQN,1
2,VIPFVASVAAEMMQHVYCAASKKC,1
3,ILGPVISTIGGVLGGLLKNL,1
4,FLWGLIPGAISAVTSLIKK,1
...,...,...
1407,GWKKWFNRAKKVGKTVGGLAVDHY,1
1408,GLFSKFAGKGIKNFLIKGVKHIGKEVGMDVIRVGIDVAGCKIKGVC,1
1409,GKWMSLLKHWLK,1
1410,IWLTALKFLGKHAAKHLAKQQLSK,1


In [17]:
df_validas = df_validas.rename(columns={'Sequence': 'sequence'})
display(df_validas.head())

,sequence,label
0,KWKKLLKKPPPLLKKLLKKL,1
1,GIWDTIKSMGKVFAGKILQN,1
2,VIPFVASVAAEMMQHVYCAASKKC,1
3,ILGPVISTIGGVLGGLLKNL,1
4,FLWGLIPGAISAVTSLIKK,1


In [18]:
df_validas.to_csv('SATPdb_labeled.csv', index=False, header=True)

## Obtener metadatos

In [19]:
import json

In [20]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [21]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [22]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [23]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "SATPdb")

In [24]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [25]:
export_json("metadata.json", dict_metadata)